In [ ]:
# one-hot encoding 
from sklearn.datasets import load_iris
import numpy as np
iris = load_iris()

x = iris.data
y = iris.target
print("x.shape:", x.shape)
print("y.shape:", y.shape)

In [ ]:
print(y)
print(iris["target_names"])

In [ ]:
# 인코딩 수행
print(y.max())
y_onehot = np.zeros((len(y), y.max()+1)) # (150, 3)
y_onehot[np.arange(len(y)), y] = 1
print(y_onehot)       

In [ ]:
def to_onehot(y):
    y_onehot = np.zeros((y.shape[0], y.max()+1)) # (150, 3)
    y_onehot[np.arange(len(y)), y] = 1
    return y_onehot

In [ ]:
# 소프트맥스 함수의 구현
%config InlineBackend.figure_format = "retina"
import matplotlib.pyplot as plt
import numpy as np
 
def softmax(z):
    exp_z = np.exp(z)
    return exp_z / np.sum(exp_z)

z = np.array([2.20, 1.39, 0.85])
y = softmax(z)
print(y)

In [ ]:
print(np.sum(y))

In [ ]:
plt.pie(y, labels=y, startangle=90)
plt.show()

In [ ]:
# 기존 소프트맥스 함수의 문제는 지수 함수 특성상 그 값이 급격하게 커져 오버플로가 발생할 수 있습니다.
# 이를 해결하는 방법은 여러 가지가 있지만 한 가지는  소프트맥스 함수로 입력된 값들 중 최대값을 찾아
# 입력된 값들과 뺄셈을 수행한 결과를 사용하여 계산하는 것입니다.

%config InlineBackend.figure_format = "retina"
import matplotlib.pyplot as plt
import numpy as np
 
def softmax(z):
    exp_z = np.exp(z - np.max(z)) # exp_z = np.exp(z)
    return exp_z / np.sum(exp_z)

z = np.array([2.20, 1.39, 0.85])
y = softmax(z)
print(y)
print(np.sum(y))

plt.pie(y, labels=y, startangle=90)
plt.show()

In [ ]:
# 붓꽃 데이터 분류 실습
from sklearn.datasets import load_iris
import numpy as np

def to_onehot(y):
    y_onehot = np.zeros((len(y), y.max()+1))
    y_onehot[np.arange(len(y)), y] = 1
    return y_onehot

def softmax(z):
    exp_z = np.exp(z - np.max(z))
    return exp_z / np.sum(exp_z, axis=1).reshape(-1,1)

def cce(a, y):
    return -np.sum(y * np.log(a))


# 1. 데이터를 준비합니다.
iris = load_iris()
x = iris.data
y = iris.target
y_onehot = to_onehot(y)

# 2. 가중치와 바이어스를 초기화합니다.
num_classes = 3
w = np.zeros((x.shape[1], num_classes)) # w(4,3)
b = np.zeros((x.shape[0], num_classes)) # b(150,3)

# 3. 학습을 수행합니다.
lr = 0.01
n_epochs = 700
for epoch in range(n_epochs+1):
    z = x.dot(w) + b
    y_hat = softmax(z)

    err = y_hat - y_onehot
    w_grad = x.T.dot(err)
    b_grad = err * 1.0

    # 가중치와 바이어스를 업데이트합니다.
    w -= lr * w_grad
    b -= lr * b_grad

    if (epoch % 100) == 0:
        err = cce(y_hat, y_onehot)
        acc = np.mean(y_hat.argmax(1) == y_onehot.argmax(1))
        print(f"epoch: {epoch:4d}/{n_epochs}, err: {err:.10f}, acc: {acc}")

In [1]:
# 케라스의 사용
# 1. 텐서의 도입
from sklearn.datasets import load_iris

import tensorflow as tf
from tensorflow.keras.utils import to_categorical # 원-핫 인코딩 함수도 제공합니다.

# 텐서플로는 softmax와 cce 함수를 제공합니다.
softmax = tf.nn.softmax
cce = tf.keras.losses.categorical_crossentropy

iris = load_iris()

x = iris.data
y = iris.target
y_onehot = to_categorical(y)

x = tf.constant(x, dtype=tf.float32)
y_onehot = tf.constant(y_onehot, dtype=tf.float32)

input_features = x.shape[1]
output_features = y_onehot.shape[1]
w = tf.Variable(tf.ones((input_features, output_features), dtype=tf.float32))
b = tf.Variable(tf.ones((len(y), output_features), dtype=tf.float32))

lr = 0.1
epochs = 3000
for epoch in range(epochs + 1):
    z = tf.matmul(x, w) + b
    y_hat = softmax(z)

    err = y_hat - y_onehot
    w_grad = tf.transpose(x) @ err
    b_grad = err * 1.0

    w.assign_sub(lr * w_grad)
    b.assign_sub(lr * b_grad)

    if (epoch % 1000) == 0:
        cost = tf.reduce_mean(cce(y_onehot, y_hat))
        print(f"epoch: {epoch:>5}/{epochs}, err: {cost:.10f}")

epoch:     0/3000, err: 1.0986123085
epoch:  1000/3000, err: 0.0000942295
epoch:  2000/3000, err: 0.0000429363
epoch:  3000/3000, err: 0.0000281090


In [2]:
pred = tf.argmax(y_hat, axis=1)
ace = tf.reduce_mean(tf.cast(pred == y, dtype=tf.float32))
print(ace.numpy())

1.0


In [3]:
# 2. 자동 미분의 도입
from sklearn.datasets import load_iris

import tensorflow as tf
from tensorflow.keras.utils import to_categorical # 원-핫 인코딩 함수도 제공합니다.

# 텐서플로는 softmax와 cce 함수를 제공합니다.
softmax = tf.nn.softmax
cce = tf.keras.losses.categorical_crossentropy

iris = load_iris()

x = iris.data
y = iris.target
y_onehot = to_categorical(y)

input_features = x.shape[1]
output_features = y_onehot.shape[1]

x = tf.constant(x, dtype=tf.float32)
y_onehot = tf.constant(y_onehot, dtype=tf.float32)

w = tf.Variable(tf.ones((input_features, output_features), dtype=tf.float32))
b = tf.Variable(tf.ones((len(y), output_features), dtype=tf.float32))

lr = 0.1
epochs = 3000
for epoch in range(epochs + 1):
    with tf.GradientTape() as tape:
        y_hat = softmax(tf.matmul(x, w) + b)
        cost = tf.reduce_mean(cce(y_onehot, y_hat))

    w_grad, b_grad = tape.gradient(cost, [w, b])

    w.assign_sub(lr * w_grad)
    b.assign_sub(lr * b_grad)

    if (epoch % 1000) == 0:
        cost = tf.reduce_mean(cce(y_onehot, y_hat))
        print(f"epoch: {epoch:>5}/{epochs}, err: {cost:.10f}")

epoch:     0/3000, err: 1.0986123085
epoch:  1000/3000, err: 0.1013638079
epoch:  2000/3000, err: 0.0619733892
epoch:  3000/3000, err: 0.0446211025


In [4]:
pred = tf.argmax(y_hat, axis=1)
ace = tf.reduce_mean(tf.cast(pred == y, dtype=tf.float32))
print(ace.numpy())

1.0


In [5]:
# 3. 옵티마이저의 도입
from sklearn.datasets import load_iris

import tensorflow as tf
from tensorflow.keras.utils import to_categorical # 원-핫 인코딩 함수도 제공합니다.

# 텐서플로는 softmax와 cce 함수를 제공합니다.
softmax = tf.nn.softmax
cce = tf.keras.losses.categorical_crossentropy

iris = load_iris()

x = iris.data
y = iris.target
y_onehot = to_categorical(y)

input_features = x.shape[1]
output_features = y_onehot.shape[1]

x = tf.constant(x, dtype=tf.float32)
y_onehot = tf.constant(y_onehot, dtype=tf.float32)

optimizer  = tf.keras.optimizers.SGD(learning_rate=0.1) # lr = 0.1
epochs = 3000
for epoch in range(epochs + 1):
    with tf.GradientTape() as tape:
        y_hat = softmax(tf.matmul(x, w) + b)
        cost = tf.reduce_mean(cce(y_onehot, y_hat))

    w_grad, b_grad = tape.gradient(cost, [w, b])

    # w.assign_sub(lr * w_grad)
    # b.assign_sub(lr * b_grad)
    optimizer.apply_gradients(zip([w_grad, b_grad], [w, b]))

    if (epoch % 1000) == 0:
        cost = tf.reduce_mean(cce(y_onehot, y_hat))
        print(f"epoch: {epoch:>5}/{epochs}, err: {cost:.10f}")

epoch:     0/3000, err: 0.0446084961
epoch:  1000/3000, err: 0.0346902534
epoch:  2000/3000, err: 0.0282898527
epoch:  3000/3000, err: 0.0238365922


In [6]:
pred = tf.argmax(y_hat, axis=1)
ace = tf.reduce_mean(tf.cast(pred == y, dtype=tf.float32))
print(ace.numpy())

1.0


In [7]:
# 4. 케라스의 사용
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.utils import to_categorical

iris = load_iris()
x = iris.data
y = iris.target
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, random_state=42)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

input_ = tf.keras.layers.Input(shape=[4])
output_ = tf.keras.layers.Dense(3, activation='softmax')(input_)

model = tf.keras.models.Model(input_, output_)
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
              loss='categorical_crossentropy', metrics=["categorical_accuracy"])

model.fit(x_train, y_train, epochs=3000)

Epoch 1/3000
4/4 [==============================] - 0s 896us/step - loss: 2.8354 - categorical_accuracy: 0.2857
Epoch 2/3000
4/4 [==============================] - 0s 849us/step - loss: 0.9965 - categorical_accuracy: 0.6875
Epoch 3/3000
4/4 [==============================] - 0s 761us/step - loss: 0.9260 - categorical_accuracy: 0.5804
Epoch 4/3000
4/4 [==============================] - 0s 715us/step - loss: 0.6854 - categorical_accuracy: 0.7500
Epoch 5/3000
4/4 [==============================] - 0s 733us/step - loss: 0.6006 - categorical_accuracy: 0.7232
Epoch 6/3000
4/4 [==============================] - 0s 585us/step - loss: 0.9097 - categorical_accuracy: 0.6607
Epoch 7/3000
4/4 [==============================] - 0s 678us/step - loss: 0.6158 - categorical_accuracy: 0.7500
Epoch 8/3000
4/4 [==============================] - 0s 664us/step - loss: 0.5832 - categorical_accuracy: 0.7589
Epoch 9/3000
4/4 [==============================] - 0s 704us/step - loss: 0.7638 - categorical_accuracy:

4/4 [==============================] - 0s 1ms/step - loss: 0.2066 - categorical_accuracy: 0.9732
Epoch 74/3000
4/4 [==============================] - 0s 648us/step - loss: 0.2136 - categorical_accuracy: 0.9375
Epoch 75/3000
4/4 [==============================] - 0s 774us/step - loss: 0.2337 - categorical_accuracy: 0.9375
Epoch 76/3000
4/4 [==============================] - 0s 669us/step - loss: 0.2275 - categorical_accuracy: 0.9554
Epoch 77/3000
4/4 [==============================] - 0s 914us/step - loss: 0.3366 - categorical_accuracy: 0.8304
Epoch 78/3000
4/4 [==============================] - 0s 806us/step - loss: 0.2038 - categorical_accuracy: 0.9643
Epoch 79/3000
4/4 [==============================] - 0s 739us/step - loss: 0.2415 - categorical_accuracy: 0.9196
Epoch 80/3000
4/4 [==============================] - 0s 838us/step - loss: 0.1986 - categorical_accuracy: 0.9554
Epoch 81/3000
4/4 [==============================] - 0s 816us/step - loss: 0.2692 - categorical_accuracy: 0.8929

4/4 [==============================] - 0s 776us/step - loss: 0.1632 - categorical_accuracy: 0.9464
Epoch 146/3000
4/4 [==============================] - 0s 803us/step - loss: 0.1591 - categorical_accuracy: 0.9732
Epoch 147/3000
4/4 [==============================] - 0s 790us/step - loss: 0.1548 - categorical_accuracy: 0.9554
Epoch 148/3000
4/4 [==============================] - 0s 864us/step - loss: 0.1529 - categorical_accuracy: 0.9643
Epoch 149/3000
4/4 [==============================] - 0s 748us/step - loss: 0.1505 - categorical_accuracy: 0.9821
Epoch 150/3000
4/4 [==============================] - 0s 740us/step - loss: 0.1496 - categorical_accuracy: 0.9643
Epoch 151/3000
4/4 [==============================] - 0s 772us/step - loss: 0.1502 - categorical_accuracy: 0.9732
Epoch 152/3000
4/4 [==============================] - 0s 693us/step - loss: 0.1875 - categorical_accuracy: 0.9464
Epoch 153/3000
4/4 [==============================] - 0s 655us/step - loss: 0.1610 - categorical_accura

4/4 [==============================] - 0s 714us/step - loss: 0.1390 - categorical_accuracy: 0.9643
Epoch 218/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1277 - categorical_accuracy: 0.9821
Epoch 219/3000
4/4 [==============================] - 0s 952us/step - loss: 0.1280 - categorical_accuracy: 0.9643
Epoch 220/3000
4/4 [==============================] - 0s 772us/step - loss: 0.1536 - categorical_accuracy: 0.9554
Epoch 221/3000
4/4 [==============================] - 0s 831us/step - loss: 0.1299 - categorical_accuracy: 0.9643
Epoch 222/3000
4/4 [==============================] - 0s 757us/step - loss: 0.1274 - categorical_accuracy: 0.9732
Epoch 223/3000
4/4 [==============================] - 0s 637us/step - loss: 0.1368 - categorical_accuracy: 0.9554
Epoch 224/3000
4/4 [==============================] - 0s 971us/step - loss: 0.1268 - categorical_accuracy: 0.9643
Epoch 225/3000
4/4 [==============================] - 0s 798us/step - loss: 0.1294 - categorical_accuracy

4/4 [==============================] - 0s 661us/step - loss: 0.1200 - categorical_accuracy: 0.9643
Epoch 290/3000
4/4 [==============================] - 0s 816us/step - loss: 0.1297 - categorical_accuracy: 0.9732
Epoch 291/3000
4/4 [==============================] - 0s 928us/step - loss: 0.1141 - categorical_accuracy: 0.9643
Epoch 292/3000
4/4 [==============================] - 0s 877us/step - loss: 0.1182 - categorical_accuracy: 0.9732
Epoch 293/3000
4/4 [==============================] - 0s 925us/step - loss: 0.1187 - categorical_accuracy: 0.9554
Epoch 294/3000
4/4 [==============================] - 0s 899us/step - loss: 0.1137 - categorical_accuracy: 0.9732
Epoch 295/3000
4/4 [==============================] - 0s 911us/step - loss: 0.1146 - categorical_accuracy: 0.9643
Epoch 296/3000
4/4 [==============================] - 0s 665us/step - loss: 0.1235 - categorical_accuracy: 0.9643
Epoch 297/3000
4/4 [==============================] - 0s 851us/step - loss: 0.1420 - categorical_accura

...

4/4 [==============================] - 0s 658us/step - loss: 0.0711 - categorical_accuracy: 0.9643
Epoch 2712/3000
4/4 [==============================] - 0s 722us/step - loss: 0.0719 - categorical_accuracy: 0.9732
Epoch 2713/3000
4/4 [==============================] - 0s 731us/step - loss: 0.0672 - categorical_accuracy: 0.9732
Epoch 2714/3000
4/4 [==============================] - 0s 659us/step - loss: 0.0678 - categorical_accuracy: 0.9643
Epoch 2715/3000
4/4 [==============================] - 0s 610us/step - loss: 0.0659 - categorical_accuracy: 0.9821
Epoch 2716/3000
4/4 [==============================] - 0s 720us/step - loss: 0.0707 - categorical_accuracy: 0.9643
Epoch 2717/3000
4/4 [==============================] - 0s 818us/step - loss: 0.0661 - categorical_accuracy: 0.9732
Epoch 2718/3000
4/4 [==============================] - 0s 686us/step - loss: 0.0720 - categorical_accuracy: 0.9643
Epoch 2719/3000
4/4 [==============================] - 0s 828us/step - loss: 0.0679 - categorica

4/4 [==============================] - 0s 721us/step - loss: 0.0696 - categorical_accuracy: 0.9732
Epoch 2783/3000
4/4 [==============================] - 0s 790us/step - loss: 0.0668 - categorical_accuracy: 0.9732
Epoch 2784/3000
4/4 [==============================] - 0s 996us/step - loss: 0.0682 - categorical_accuracy: 0.9732
Epoch 2785/3000
4/4 [==============================] - 0s 698us/step - loss: 0.0662 - categorical_accuracy: 0.9732
Epoch 2786/3000
4/4 [==============================] - 0s 876us/step - loss: 0.0663 - categorical_accuracy: 0.9732
Epoch 2787/3000
4/4 [==============================] - 0s 781us/step - loss: 0.0681 - categorical_accuracy: 0.9732
Epoch 2788/3000
4/4 [==============================] - 0s 764us/step - loss: 0.0766 - categorical_accuracy: 0.9643
Epoch 2789/3000
4/4 [==============================] - 0s 672us/step - loss: 0.0691 - categorical_accuracy: 0.9821
Epoch 2790/3000
4/4 [==============================] - 0s 893us/step - loss: 0.0685 - categorica

4/4 [==============================] - 0s 643us/step - loss: 0.0728 - categorical_accuracy: 0.9732
Epoch 2854/3000
4/4 [==============================] - 0s 742us/step - loss: 0.0681 - categorical_accuracy: 0.9643
Epoch 2855/3000
4/4 [==============================] - 0s 912us/step - loss: 0.0656 - categorical_accuracy: 0.9732
Epoch 2856/3000
4/4 [==============================] - 0s 731us/step - loss: 0.0757 - categorical_accuracy: 0.9643
Epoch 2857/3000
4/4 [==============================] - 0s 879us/step - loss: 0.0813 - categorical_accuracy: 0.9643
Epoch 2858/3000
4/4 [==============================] - 0s 760us/step - loss: 0.0741 - categorical_accuracy: 0.9821
Epoch 2859/3000
4/4 [==============================] - 0s 860us/step - loss: 0.0675 - categorical_accuracy: 0.9732
Epoch 2860/3000
4/4 [==============================] - 0s 796us/step - loss: 0.0689 - categorical_accuracy: 0.9643
Epoch 2861/3000
4/4 [==============================] - 0s 747us/step - loss: 0.0664 - categorica

4/4 [==============================] - 0s 828us/step - loss: 0.0645 - categorical_accuracy: 0.9821
Epoch 2925/3000
4/4 [==============================] - 0s 737us/step - loss: 0.0645 - categorical_accuracy: 0.9732
Epoch 2926/3000
4/4 [==============================] - 0s 764us/step - loss: 0.0654 - categorical_accuracy: 0.9732
Epoch 2927/3000
4/4 [==============================] - 0s 796us/step - loss: 0.0670 - categorical_accuracy: 0.9732
Epoch 2928/3000
4/4 [==============================] - 0s 862us/step - loss: 0.0665 - categorical_accuracy: 0.9732
Epoch 2929/3000
4/4 [==============================] - 0s 742us/step - loss: 0.0701 - categorical_accuracy: 0.9732
Epoch 2930/3000
4/4 [==============================] - 0s 968us/step - loss: 0.0694 - categorical_accuracy: 0.9643
Epoch 2931/3000
4/4 [==============================] - 0s 684us/step - loss: 0.0681 - categorical_accuracy: 0.9732
Epoch 2932/3000
4/4 [==============================] - 0s 699us/step - loss: 0.0705 - categorica

4/4 [==============================] - 0s 687us/step - loss: 0.0692 - categorical_accuracy: 0.9732
Epoch 2996/3000
4/4 [==============================] - 0s 682us/step - loss: 0.0661 - categorical_accuracy: 0.9643
Epoch 2997/3000
4/4 [==============================] - 0s 870us/step - loss: 0.0721 - categorical_accuracy: 0.9643
Epoch 2998/3000
4/4 [==============================] - 0s 683us/step - loss: 0.0703 - categorical_accuracy: 0.9732
Epoch 2999/3000
4/4 [==============================] - 0s 768us/step - loss: 0.0652 - categorical_accuracy: 0.9643
Epoch 3000/3000
4/4 [==============================] - 0s 706us/step - loss: 0.0672 - categorical_accuracy: 0.9643


In [8]:
model.evaluate(x_test, y_test)

2/2 [==============================] - 0s 963us/step - loss: 0.0516 - categorical_accuracy: 1.0000


[0.051603976637125015, 1.0]